<a href="https://colab.research.google.com/github/kyle-gao/TF_Transformer/blob/master/BLEUScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2020 Yi Lin(Kyle) Gao


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [90]:
import nltk
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

The validation data consists of the first 25 lines of the 2000-05-09 senate debate (part of the validation set, not used for training) of the Hansards dataset.

In [3]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.load_from_file("Data/tokenizer_en")
tokenizer_fr = tfds.features.text.SubwordTextEncoder.load_from_file("Data/tokenizer_fr")
en_vocab_size = tokenizer_en.vocab_size + 2
fr_vocab_size = tokenizer_fr.vocab_size + 2

In [117]:
def positional_encoding(pos, d_model):
    """
    :param pos: int max position
    :param d_model: dimension of the model
    :return: (1,pos,d_model) array of sinusoidal positional encoding
    """
    pos_enc = np.zeros((1, pos, d_model))
    for p in range(pos):
        for i in range(d_model // 2):
            angles = p / np.power(10000, (2 * i) / np.float32(d_model))
            pos_enc[:, p, 2 * i] = np.sin(angles)
            pos_enc[:, p, 2 * i + 1] = np.cos(angles)
        if d_model % 2 == 1:
            # if d_model is odd loop doesn't hit last even index
            angles = p / np.power(10000, (2 * d_model) / np.float32(d_model))
            pos_enc[:, p, d_model - 1] = np.sin(angles)
    return tf.cast(pos_enc, tf.float32)

class MultiHeadAttention(tf.keras.layers.Layer):

    """Implemented with tf.einsum(), is faster than using tf.transpose() with tf.matmul()"""

    def __init__(self, d_model, num_heads):
        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads,depth)

        Arguments:
        x -- A tokenized sequence (batch_size, seq_len, d_model)

        Returns:
        A tokenized sequence with dimensions (batch_size, seq_len, num_heads, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))

        return x

    def call(self, q, k, v, mask=None):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)  # (batch_size,len_q, dim_q)
        k = self.wk(k)  # (batch_size,len_v, dim_q)
        v = self.wv(v)  # (batch_size,len_v, dim_v)

        q = self.split_heads(q, batch_size)  # (batch_size, len_q, num_heads, depth_q) (m,l,h,d)
        k = self.split_heads(k, batch_size)  # (batch_size, len_v, num_heads, depth_q) (m,j,h,d)
        v = self.split_heads(v, batch_size)  # (batch_size, len_v, num_heads, depth_v) (m,j,h,e)

        qk = tf.einsum("mlhd,mjhd->mljh", q, k)  # (batch_size, len_q, len_v, num_heads) (m,l,j,h)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        qk = qk / tf.math.sqrt(dk)

        if mask is not None:
            qk = qk - mask*1e9 # We are using a multiplicative mask

        qk = tf.nn.softmax(qk, axis=-2)  # (batch_size,len_q,len_v, num_heads) (m,l,j,h)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        qk = qk / tf.math.sqrt(dk)

        output = tf.einsum("mljh, mjhe -> mlhe", qk, v)  # (batch_size,len_q, heads, depth_v)
        output = tf.reshape(output, (batch_size, -1, self.num_heads * self.depth))  # (batch_size,len_q, d_model)

        return self.dense(output)

class EncoderLayer(tf.keras.layers.Layer):
    """The EncoderLayer consists of one MultiHeadAttention layer connected to a FeedForward layer,
    each of these 2 layers have a residual connection."""

    def __init__(self, num_heads, d_model, dense_dim, dropout=0.1):
        super().__init__()

        self.attention = MultiHeadAttention(d_model, num_heads)
        self.dense = tf.keras.Sequential([tf.keras.layers.Dense(dense_dim, activation='relu'),
                                          tf.keras.layers.Dense(d_model)])

        self.norm1 = tf.keras.layers.LayerNormalization()
        self.norm2 = tf.keras.layers.LayerNormalization()

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, x, training, mask):
        out_attention = self.attention(x, x, x, mask)  # (batch_size,seq_len,d_model)
        out_attention = self.dropout1(out_attention, training=training)
        out1 = self.norm1(x + out_attention)  # residual connection (batch_size,seq_len,d_model)

        out_dense = self.dense(out1)  # (batch_size,seq_len,d_model)
        out2 = self.norm2(out1 + out_dense)  # residual conenction (batch_size,seq_len,d_model)
        return out2


class Encoder(tf.keras.layers.Layer):
    """The Encoder consists of EncoderLayer"""

    def __init__(self, num_layers, num_heads, d_model, dense_dim,
                 vocab_size, max_encoding_position, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.positional_encoding = positional_encoding(max_encoding_position, d_model)
        self.encoding_layers = [EncoderLayer(num_heads, d_model, dense_dim, dropout) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, training, mask=None):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)  # (batch_size,input_len,d_model)
        x = x * tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.positional_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.encoding_layers[i](x, training, mask)  # (batch_size, input_seq_len, d_model)

        return x
class DecoderLayer(tf.keras.layers.Layer):
    """A decoder layers consists of two MultiHeadAttention, one for the Decoder input, one from Encoder output"""
    def __init__(self, num_heads, d_model, dense_dim, dropout=0.1):
        super().__init__()

        self.attention1 = MultiHeadAttention(d_model, num_heads)
        self.attention2 = MultiHeadAttention(d_model, num_heads)

        self.dense = tf.keras.Sequential([tf.keras.layers.Dense(dense_dim, activation='relu'),
                                          tf.keras.layers.Dense(d_model)])

        self.norm1 = tf.keras.layers.LayerNormalization()
        self.norm2 = tf.keras.layers.LayerNormalization()
        self.norm3 = tf.keras.layers.LayerNormalization()

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)
        self.dropout3 = tf.keras.layers.Dropout(dropout)

    def call(self, encoder_out, x, training, forward_mask, padding_mask):

        out_attention1 = self.attention1(x, x, x,
                                         forward_mask)  # (batch_size, seq_len_answer, d_model) -> The return seq_len is the same as that of the first argument of the call.
        out_attention1 = self.dropout1(out_attention1, training=training)
        out1 = self.norm1(x + out_attention1)  # residual connection (batch_size, seq_len_answer, d_model)

        out_attention2 = self.attention2(out1, encoder_out, encoder_out,
                                         padding_mask)  # (batch_size, seq_len_answer, d_model)
        out_attention2 = self.dropout2(out_attention2, training=training)
        out2 = self.norm2(out1 + out_attention2)

        out_dense = self.dense(out2)
        out_dense = self.dropout3(out_dense + out2)

        return out_dense


class Decoder(tf.keras.layers.Layer):
    """The Decoder consists of multiple DecoderLayer"""
    def __init__(self, num_layers, num_heads, d_model, dense_dim,
                 vocab_size, max_encoding_position, dropout=0.1):
        super().__init__()

        self.num_heads = num_heads
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.positional_encoding = positional_encoding(max_encoding_position, d_model)
        self.decoder_layers = [DecoderLayer(num_heads, d_model, dense_dim, dropout) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, encoder_out, x, training, forward_mask=None, padding_mask=None):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)  # (batch_size,input_len,d_model)
        x = x * tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.positional_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.decoder_layers[i](encoder_out, x, training, forward_mask,
                                       padding_mask)  # (batch_size, input_seq_len, d_model)
        return x

class Transformer(tf.keras.Model):

    def __init__(self, num_layers, num_heads, d_model, dense_dim, in_vocab_size, tar_vocab_size,
                 input_max_position, target_max_position, rate=0.1):
        super().__init__()

        self.encoder = Encoder(num_layers, num_heads, d_model, dense_dim,
                               in_vocab_size, max_encoding_position=input_max_position, dropout=0.1)

        self.decoder = Decoder(num_layers, num_heads, d_model, dense_dim,
                               tar_vocab_size, max_encoding_position=target_max_position, dropout=0.1)

        self.dense = tf.keras.layers.Dense(tar_vocab_size)

    def call(self, input, target, training=False, enc_mask=None, dec_forward_mask=None, dec_padding_mask=None):
        out_encoder = self.encoder(input, training=training, mask=enc_mask)

        out_decoder = self.decoder(out_encoder, target, training=training, forward_mask=dec_forward_mask,
                                   padding_mask=dec_padding_mask)

        out = self.dense(out_decoder)

        return out


def padding_mask(seq):
    # Returns (batch, seq_len, 1, 1) tensor with 0's where the sequence is padded, 1 where it is not

    mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return mask[:, tf.newaxis, :,  tf.newaxis]  # (batch, 1, seq_len, 1) m l j h  <- j gets masked


def forward_mask(seq):
    """
    Calculates a combined forward mask and padding mask for a batch of sequences
    :param seq: (batch,seq_len) a batch of sequences
    :return:  a combined look_ahead_mask (lower triangular 1s)
    and padding mask (batch, seq_len, seq_len, 1)
    """
    seq_len = tf.shape(seq)[1]

    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    look_ahead_mask = look_ahead_mask[tf.newaxis, :, :, tf.newaxis]  # (batch, seq_len, seq_len, 1)

    padded_mask = padding_mask(seq)

    # return padded_mask * look_ahead_mask  # (batch, seq_len, seq_len, 1)
    return tf.maximum(padded_mask, look_ahead_mask)

    
def evaluate(question):

    start_token = [tokenizer_fr.vocab_size]
    end_token = [tokenizer_fr.vocab_size + 1]
    question = start_token + tokenizer_fr.encode(question) + end_token
    question = tf.expand_dims(question, 0)
    answer_in = [tokenizer_en.vocab_size]
    answer_in = tf.expand_dims(answer_in, 0)

    for i in range(max_len_fr):
        enc_padding_mask = padding_mask(question)
        dec_padding_mask = padding_mask(question)
        dec_forward_mask = forward_mask(answer_in)

        predictions = transformer(question, answer_in, training=False, enc_mask=enc_padding_mask,
                                  dec_forward_mask=dec_forward_mask, dec_padding_mask=dec_padding_mask)
        prediction = predictions[:, -1, :]  # select the last word to add to the outputs

        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)

        if predicted_id == tokenizer_en.vocab_size + 1:
            return tf.squeeze(answer_in, axis=0)
        predicted_id = tf.expand_dims(predicted_id, 0)
        answer_in = tf.concat([answer_in, predicted_id], axis=-1)

    return tf.squeeze(answer_in, axis=0)


def translate(sentence):
    result = np.array(evaluate(sentence))

    predicted_sentence = tokenizer_en.decode([i for i in result
                                              if tokenizer_en.vocab_size > i > 0])
    print('Input: {}'.format(sentence))
    print('Predicted answer: {}'.format(predicted_sentence))
    return predicted_sentence

In [25]:
ref1_path = "Data/Val/SenateDebate2000-05-09_e.txt"
ref2_path = "Data/Val/SenateDebate2000-05-09_GoogleTranslate.txt"
val_path = "Data/Val/SenateDebate2000-05-09_f.txt"

In [124]:
def process_string (path):
  with open(path) as file:
    str_lst = file.readlines()
  str_lst = str_lst[:25]
  str_lst = [s.rstrip("\n") for s in str_lst]
  return str_lst
val_data = process_string(val_path)
ref1 = process_string(ref1_path)
ref2 = process_string(ref2_path)


In [125]:
ref1 = [s.split(" ") for s in ref1]
ref2 = [s.split(" ") for s in ref2]

In [94]:
num_layers = 4
d_model = 128
dense_dim = 256
num_heads = 8


max_len_en = 50
min_len_en = 10  # The transcript has many short lines indicating the date or speaker, which we should filter out.
max_len_fr = 50
min_len_fr = 10

transformer = Transformer(num_layers=num_layers, num_heads=num_heads, d_model=d_model, dense_dim=dense_dim,
                          in_vocab_size=fr_vocab_size, tar_vocab_size=en_vocab_size,
                          input_max_position=max_len_fr, target_max_position=max_len_en, rate=0.1)
transformer.load_weights("transformer")

In [126]:
translation = [translate(s) for s in val_data]
translation=[s.split(" ") for s in translation]

Input: debats du senat (hansard) 
Predicted answer: senate debates are scandal 
Input: 2e session, 36e legislature, 
Predicted answer: 2 the 27th session of parliament is a day of parliament 
Input: volume 138, numero 53 
Predicted answer: the 138883 will be 53 
Input: le mardi 9 mai 2000 
Predicted answer: on tuesday may 9 2000 
Input: l'honorable gildas l. molgat, president 
Predicted answer: the honourable gildas l molgat president 
Input: table des matieres 
Predicted answer: table material 
Input: declarations de senateurs 
Predicted answer: statements of senators are made by senators 
Input: la seconde guerre mondiale 
Predicted answer: second world war ii 
Input: le cinquante-cinquieme anniversaire du jour de la victoire en europe 
Predicted answer: fiftyfifth anniversary of victory in europe 
Input: la semaine nationale des soins palliatifs 
Predicted answer: national palliative care week 
Input: le deces du juge ronald newton pugsley, c.r. 
Predicted answer: the late justice r

Corpus Bleu reference vs google translate

In [130]:
hypothesis = ref2 #list(corpus) of list(setence) of strings(words)
reference = [[s] for s in ref1] #list(corpus) of list of list (list of reference sentences) of strings
print(nltk.translate.bleu_score.corpus_bleu(reference, hypothesis)) #corpus bleu for google translation

0.317355540283643


Corpus Bleu reference vs Transformer translation

In [138]:
hypothesis = translation #list(corpus) of list(setence) of strings(words)
reference = [[s] for s in ref1] #list(corpus) of list of list (list of reference sentences) of strings
print(nltk.translate.bleu_score.corpus_bleu(reference, hypothesis)) #corpus bleu for google translation

0.21291744785409664


Corpus Bleu score using both human and google translation as references

In [140]:

hypothesis = translation #list(corpus) of list(setence) of strings(words)
reference = [[ref1[i], ref2[i]] for i in range(len(ref1))] #list(corpus) of list of list (list of reference sentences) of strings
print(nltk.translate.bleu_score.corpus_bleu(reference, hypothesis)) #corpus bleu for google translation

0.301703545283996
